# Results

In [5]:
# # Import CSV from 
# # results/grayscale_sun/per_image_metrics.csv
# # results/canny_sun/per_image_metrics.csv
# # results/LoG_sun/per_image_metrics.csv
# # results/random_sun/per_image_metrics.csv

# import pandas as pd
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

In [6]:
# def load_csv(file_path):
#     """
#     Load CSV file and return DataFrame.
#     """
#     if os.path.exists(file_path):
#         df = pd.read_csv(file_path)
#         return df
#     else:
#         print(f"File {file_path} does not exist.")
#         return None
    
#     # metric,mean,std
# def plot_metrics(df, title):
#     """
#     Plot metrics from DataFrame.
#     """
#     plt.figure(figsize=(10, 6))
#     sns.barplot(x='metric', y='mean', data=df, yerr=df['std'], capsize=.2)
#     plt.title(title)
#     plt.xlabel('Metric')
#     plt.ylabel('Value')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()

In [1]:
# # import the 4 csv files
# experiment_name = 'lapa'
# grayscale_df = load_csv('../results/grayscale_{}/per_image_metrics.csv'.format(experiment_name))
# canny_df = load_csv('../results/canny_{}/per_image_metrics.csv'.format(experiment_name))
# log_df = load_csv('../results/DoG_{}/per_image_metrics.csv'.format(experiment_name))
# random_df = load_csv('../results/random_{}/per_image_metrics.csv'.format(experiment_name))

# # Combine the dataframes
# combined_df = pd.concat([grayscale_df, canny_df, log_df, random_df], keys=['Grayscale', 'Canny', 'DoG', 'Random'])
# # Reset index
# combined_df.reset_index(level=0, inplace=True)
# combined_df.rename(columns={'level_0': 'Method'}, inplace=True)

In [2]:
# def calculate_mean_sem(df):
#     """
#     Calculate mean and standard error of the mean (sem) for each metric.
#     """
#     metrics = ['mse', 'ssim', 'vgg_perceptual', 'lpips', 'fsim', 'dists', 'pieapp', 'mdsi', 'vsi', 'srsim', 'infer_time_s']
#     results = []
    
#     for metric in metrics:
#         mean = df[metric].mean()
#         sem = df[metric].std() / np.sqrt(len(df))
#         results.append({'metric': metric, 'mean': mean, 'sem': sem})
    
#     return pd.DataFrame(results)

# # Calculate mean and sem for each method
# grayscale_metrics = calculate_mean_sem(grayscale_df)
# canny_metrics = calculate_mean_sem(canny_df)
# log_metrics = calculate_mean_sem(log_df)
# random_metrics = calculate_mean_sem(random_df)
# # Combine the metrics into a single DataFrame
# combined_metrics = pd.concat([grayscale_metrics, canny_metrics, log_metrics, random_metrics], keys=['Grayscale', 'Canny', 'DoG', 'Random'])
# # give a column name for the method
# combined_metrics.reset_index(level=0, inplace=True)
# combined_metrics.rename(columns={'level_0': 'method'}, inplace=True)


In [3]:
# metrics = combined_metrics['metric'].unique()
# n = len(metrics)
# cols = 2
# rows = (n + 1)//cols

# fig, axes = plt.subplots(rows, cols, figsize=(8*cols, 5*rows))
# axes = axes.flatten()

# for ax, metric in zip(axes, metrics):
#     dfm = combined_metrics[combined_metrics['metric'] == metric]
#     ax.bar(dfm['method'], dfm['mean'], 
#            yerr=dfm['sem'], 
#            capsize=4)
#     ax.set_title(metric)
#     ax.set_ylabel('Score')
#     ax.set_xticklabels(dfm['method'], rotation=45, ha='right')
    
# # hide any unused axes
# for ax in axes[len(metrics):]:
#     ax.axis('off')

# # title is dataset
# plt.suptitle(f'Metrics for {experiment_name} dataset', fontsize=16)

# plt.tight_layout()
# plt.show()


In [5]:
# # --- Imports and config ---
# import os, math, json
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from pathlib import Path
# from scipy import stats

# sns.set_context("talk")

# # Utility: safe load
# def load_csv(fp):
#     if os.path.exists(fp):
#         return pd.read_csv(fp)
#     print(f"[warn] Missing: {fp}")
#     return None

# # Where your results live
# experiment_name = 'lapa'   # e.g. 'sun', 'lapa', 'coco'
# base = Path("../results")

# # Optionally support multiple seeds per method: results/<method>_<dataset>/seed<N>/per_image_metrics.csv
# def gather_method(method_tag, dataset, seeds=None):
#     dfs = []
#     if seeds is None:
#         df = load_csv(str(base / f"{method_tag}_{dataset}" / "per_image_metrics.csv"))
#         if df is not None:
#             df["seed"] = 0
#             dfs.append(df)
#     else:
#         for s in seeds:
#             df = load_csv(str(base / f"{method_tag}_{dataset}" / f"seed{s}" / "per_image_metrics.csv"))
#             if df is not None:
#                 df["seed"] = s
#                 dfs.append(df)
#     if len(dfs) == 0:
#         return None
#     out = pd.concat(dfs, ignore_index=True)
#     out["method"] = method_tag.capitalize().replace("dog","DoG")
#     return out

# # Choose seeds if you have them. Otherwise set to None.
# seeds = None  # e.g. [0,1,2,3,4]
# methods = ["grayscale","canny","DoG","random"]

# dfs = []
# for m in methods:
#     d = gather_method(m, experiment_name, seeds)
#     if d is not None:
#         dfs.append(d)

# assert len(dfs) >= 2, "Need at least two methods loaded"
# df_all = pd.concat(dfs, ignore_index=True)

# # If PSNR is not present, compute from MSE if you stored image dynamic range in your CSV
# # Otherwise, skip gracefully
# if "psnr" not in df_all.columns and "mse" in df_all.columns:
#     # Assume images normalized to [0,1]
#     df_all["psnr"] = -10*np.log10(df_all["mse"].clip(0))

# # Keep a canonical metric list
# metric_cols = [c for c in ["mse","ssim","psnr","fsim","lpips","dists","pieapp","mdsi","vsi","srsim","vgg_perceptual","infer_time_s"] if c in df_all.columns]

# # Sanity: per-image merge key
# merge_keys = [c for c in ["image_id","filename","name"] if c in df_all.columns]
# if len(merge_keys) == 0:
#     # manufacture an image index per method group to enable pairing
#     df_all["image_idx"] = df_all.groupby("method").cumcount()
#     merge_keys = ["image_idx"]


In [6]:
# def mean_ci(x, alpha=0.05):
#     x = np.asarray(x); x = x[~np.isnan(x)]
#     n = len(x)
#     if n == 0:
#         return np.nan, np.nan, np.nan
#     m = x.mean()
#     se = x.std(ddof=1) / math.sqrt(n)
#     t = stats.t.ppf(1 - alpha/2, df=max(n-1,1))
#     return m, m - t*se, m + t*se

# # For ranking we keep metric direction. Define whether higher is better.
# higher_better = {
#     "mse": False, "ssim": False, "psnr": True, "fsim": False,
#     "lpips": False, "dists": False, "pieapp": False, "mdsi": False,
#     "vsi": False, "srsim": False, "vgg_perceptual": False,  # VGG distance
#     "infer_time_s": False
# }

# summary_rows = []
# for met in metric_cols:
#     tmp = df_all.groupby("method")[met].apply(list).to_dict()
#     for meth, arr in tmp.items():
#         m, lo, hi = mean_ci(arr, alpha=0.05)
#         summary_rows.append({"metric": met, "method": meth, "mean": m, "ci_low": lo, "ci_high": hi})

# summary_df = pd.DataFrame(summary_rows)

# # Rank within each metric
# def rank_rows(sub):
#     hb = higher_better[sub.name]
#     sub = sub.sort_values("mean", ascending=not hb).copy()
#     sub["rank"] = np.arange(1, len(sub)+1)
#     return sub

# ranked = summary_df.groupby("metric", group_keys=False).apply(rank_rows)
# ranked.head()


In [7]:
# from itertools import combinations

# def paired_stats(df, metric, methods, keys):
#     # pivot per image for pairing
#     sub = df[df["method"].isin(methods)][keys + ["method", metric]].copy()
#     piv = sub.pivot_table(index=keys, columns="method", values=metric)
#     piv = piv.dropna(axis=0)  # drop images missing any method

#     pairs = []
#     for a,b in combinations(methods, 2):
#         xa = piv[a].values
#         xb = piv[b].values
#         t, p = stats.ttest_rel(xa, xb, nan_policy="omit")
#         # Cohen's dz for paired design
#         d = (xa - xb).mean() / (xa - xb).std(ddof=1)
#         pairs.append({"metric": metric, "A": a, "B": b, "t": t, "p_raw": p, "dz": d, "n": len(xa)})
#     out = pd.DataFrame(pairs)
#     # Holm correction
#     out = out.sort_values("p_raw").reset_index(drop=True)
#     m = len(out)
#     out["p_holm"] = [min(1.0, out.loc[i,"p_raw"]*(m-i)) for i in range(m)]
#     return out

# tests = []
# for met in metric_cols:
#     tests.append(paired_stats(df_all, met, sorted(df_all["method"].unique()), merge_keys))
# tests_df = pd.concat(tests, ignore_index=True)
# tests_df.head()


In [1]:
import os, math
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats

# ---------------------------------------------------------------------
# Config
# ---------------------------------------------------------------------
DATASETS = ["lapa", "sun", "coco"]          # change as needed
METHODS  = ["DoG", "canny", "random"]  # folder prefixes

# Metric groups for the table header
INTENSITY_METRICS  = ["mse", "ssim", "psnr"]
PERCEPTUAL_METRICS = ["lpips", "dists", "mdsi", "vsi"]  # add "vgg_perceptual" if you log it

# Direction of improvement
HIGHER_BETTER = {
    "mse": False, "ssim": False, "psnr": True, "fsim": False,
    "lpips": False, "dists": False, "pieapp": False, "mdsi": False,
    "vsi": False, "srsim": False, "vgg_perceptual": False
}

# Pretty names for LaTeX header
PRETTY = {
    "mse":"MSE", "ssim":"SSIM", "psnr":"PSNR",
    "lpips":"LPIPS", "dists":"DISTS", "pieapp":"PIEAPP", "mdsi":"MDSI", "vsi":"VSI", "srsim":"SRSIM",
    "vgg_perceptual":"VGG perc."
}
PRETTY_METHOD = {
    "grayscale":"Grayscale", "canny":"Canny", "DoG":"SCAPE", "random":"Random"
}

RESULTS_DIR = Path("../results")   # adjust if needed
OUT_DIR = Path("./tables"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------
def load_per_image_metrics(method_tag, dataset):
    """Load ../results/<method>_<dataset>/per_image_metrics.csv"""
    fp = RESULTS_DIR / f"{method_tag}_{dataset}" / "per_image_metrics.csv"
    if not fp.exists():
        print(f"[warn] missing {fp}")
        return None
    df = pd.read_csv(fp)
    df["method"] = PRETTY_METHOD.get(method_tag, method_tag)
    return df

def ci95(values):
    x = np.asarray(values); x = x[~np.isnan(x)]
    n = len(x)
    if n == 0:
        return np.nan, np.nan
    m = x.mean()
    se = x.std(ddof=1) / math.sqrt(n)
    t = stats.t.ppf(0.975, df=max(n-1, 1))
    half = t * se
    return m, half

def format_entry(mean, half):
    if np.isnan(mean):
        return "--"
    return f"{mean:.3f}"

def bold_best(entries, higher_better=True):
    """entries is a list of floats (means). returns set of indices that are best"""
    arr = np.array(entries, dtype=float)
    if np.all(np.isnan(arr)):
        return set()
    if higher_better:
        best_val = np.nanmax(arr)
    else:
        best_val = np.nanmin(arr)
    # allow tiny ties
    eps = 1e-12
    return set(np.where(np.isclose(arr, best_val, atol=eps))[0].tolist())

def available_metrics(df, wanted):
    return [m for m in wanted if m in df.columns]

def build_table_for_dataset(dataset):
    # load all methods
    dfs = []
    for m in METHODS:
        d = load_per_image_metrics(m, dataset)
        if d is not None:
            dfs.append(d)
    if not dfs:
        print(f"[warn] no data for {dataset}")
        return None
    df = pd.concat(dfs, ignore_index=True)

    # derive PSNR from MSE if missing and images are in [0,1]
    if "psnr" not in df.columns and "mse" in df.columns:
        df["psnr"] = -10 * np.log10(df["mse"].clip(1e-12))

    imetrics  = available_metrics(df, INTENSITY_METRICS)
    pmetrics  = available_metrics(df, PERCEPTUAL_METRICS)
    all_metrics = imetrics + pmetrics

    # compute mean and CI per method per metric
    summary = {}
    for method_name, dsub in df.groupby("method"):
        row = {}
        for met in all_metrics:
            m, h = ci95(dsub[met].values)
            row[met] = (m, h)
        summary[method_name] = row

    # decide best per metric for bolding
    best_indices = {}
    method_list = [PRETTY_METHOD.get(m, m) for m in METHODS if PRETTY_METHOD.get(m, m) in summary]
    for met in all_metrics:
        means = [summary[m][met][0] for m in method_list]
        best_indices[met] = bold_best(means, higher_better=HIGHER_BETTER.get(met, False))

    # build LaTeX
    def header_block(title, metrics):
        cols = " ".join([PRETTY[m] for m in metrics])
        return f"  & \\multicolumn{{{len(metrics)}}}{{c}}{{{title}}} \\\\"

    def cmidrule(start_col, width):
        return f"\\cmidrule(lr){{{start_col}-{start_col+width-1}}}"

    n_int = len(imetrics)
    n_per = len(pmetrics)
    # column layout: 1 (Processing model) + n_int + n_per
    colspec = "l" + "c"*n_int + " " + "c"*n_per

    lines = []
    lines.append("\\begin{table}[t]")
    lines.append("\\centering")
    lines.append("\\small")
    lines.append(f"\\caption{{Reconstruction metrics on the {dataset.upper()} dataset. Lower is better for MSE, LPIPS, DISTS, PIEAPP, MDSI, VGG perceptual. Higher is better for SSIM, PSNR, FSIM, VSI, SRSIM. Best per column in bold.}}")
    lines.append(f"\\label{{tab:recon_{dataset}}}")
    lines.append("\\begin{tabular}{" + colspec + "}")
    lines.append("\\toprule")
    # first header row with groups
    left = "Processing model "
    right_groups = []
    if n_int > 0:
        right_groups.append(f"\\multicolumn{{{n_int}}}{{c}}{{Intensity reconstruction}}")
    if n_per > 0:
        right_groups.append(f"\\multicolumn{{{n_per}}}{{c}}{{Perceptual reconstruction}}")
    lines.append(left + " & " + " & ".join(right_groups) + " \\\\")
    # cmidrules
    start = 2
    rules = []
    if n_int > 0:
        rules.append(cmidrule(start, n_int))
        start += n_int
    if n_per > 0:
        rules.append(cmidrule(start, n_per))
    lines.append(" ".join(rules))
    # second header row with metric names
    header_metrics = [PRETTY[m] for m in imetrics + pmetrics]
    lines.append("  & " + " & ".join(header_metrics) + " \\\\")
    lines.append("\\midrule")

    # rows
    for i, method_name in enumerate(method_list):
        row = [method_name]
        for j, met in enumerate(all_metrics):
            mean, half = summary[method_name][met]
            cell = format_entry(mean, half)
            if i in best_indices[met]:
                cell = f"\\textbf{{{cell}}}"
            row.append(cell)
        lines.append(" & ".join(row) + " \\\\")
    lines.append("\\bottomrule")
    lines.append("\\end{tabular}")
    lines.append("\\end{table}")

    latex = "\n".join(lines)
    out_path = OUT_DIR / f"recon_{dataset}.tex"
    out_path.write_text(latex, encoding="utf-8")
    print(f"Wrote {out_path}")
    return latex

# ---------------------------------------------------------------------
# Run for all datasets
# ---------------------------------------------------------------------
latex_by_ds = {ds: build_table_for_dataset(ds) for ds in DATASETS}


Wrote tables/recon_lapa.tex
Wrote tables/recon_sun.tex
Wrote tables/recon_coco.tex
